# Get Line of stations from Renfe data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('..')

In [2]:
import datetime

from src.robin.scraping.renfe.entities import RenfeScraper

scraper = RenfeScraper(stations_csv_path='../data/renfe/renfe_stations.csv')

for station_id, station_name in scraper.available_stations.items():
    print(f'{station_id}: {station_name}')

?: Estaciones de Origen
31412: A Coruña
94707: Abrantes
60911: Alicante / Alacant
60600: Albacete
06008: Alcantarilla-Los Romanos
60400: Alcázar de San Juan
55020: Algeciras
56312: Almería
99003: Altet Bus
99115: Aguadulce Bus
87912: Aix En Provence
99114: Andorra-Bus
ANTEQ: Antequera (TODAS)
87814: Avignon
10400: Avila
37606: Badajoz
BARCE: Barcelona (TODAS)
87078: Beziers
65318: Benicassim
BILBA: Bilbao (TODAS)
54400: Bobadilla
11014: Burgos Rosa Manzano
35400: Cáceres
51405: Cádiz
70600: Calatayud
50417: Campus Rabanales
61307: Cartagena
65300: Castellón /Castelló
37200: Ciudad Real
50500: Córdoba
CUENC: Cuenca (TODAS)
92201: Denia-Bus
60905: Elda-Petrer
03410: Elche AV/Elx AV
94428: Entroncamento
92157: Estepona Bus
21010: Ferrol
79309: Figueres
79333: Figueres Bus
04307: Figueres Vilafant
69110: Gandía
GIJON: Gijón
79300: Girona
05000: Granada
GUADA: Guadalajara (TODAS)
43019: Huelva
74200: Huesca
IRUN-: Irun-Hendaya (TODAS)
80100: Pamplona/Iruña
99103: Jaca-Bus
03100: Jaén
64100:

In [3]:
scraper.stations_df

stop_id                       stop_name renfe_id   stop_lat  stop_lon
0    00000                         Unknown    00000   0.000000  0.000000
1    31412                       A Corunya    31412  43.352761 -8.409755
2    60911                 AlicanteAlacant    60911  38.344450 -0.495053
3    60600             Albacete-Los Llanos    60600  38.999384 -1.848450
4    60400             Alcazar de San Juan    60400  39.395628 -3.205744
..     ...                             ...      ...        ...       ...
92   13200  Bilbao-Abando Indalecio Prieto    BILBA  43.259609 -2.929150
93   66100                          Cuenca    CUENC  40.067340 -2.136471
94   15410                      GijonXixon    GIJON  43.535175 -5.698318
95   70200                     Guadalajara    GUADA  40.644103 -3.182230
96   78200                       Tardienta    78200  41.975751 -0.538314

[97 rows x 5 columns]

In [4]:
import pandas as pd 

trips = ['60000',  # MADRID
         '51405',  # CÁDIZ
         '54413',  # MÁLAGA
         '05000']  # GRANADA

"""
         '71801',
         '74200',  # HUESCA
         '15100',  # LEÓN
         '11014',  # BURGOS
         '31412',  # A CORUÑA
         '23004',  # PONTEVEDRA
         '65300',  # CASTELLÓN
         '60911']  # ALICANTE
"""

master = trips[0]
destinations = trips[1:]

df_trips, df_stops = pd.DataFrame(), pd.DataFrame()

for destination in destinations:
    for i in range(2):
        trip = None
        if i == 0:
            print(f'From {master} to {destination}')
            trip = (master, destination)
        else:
            print(f'From {destination} to {master}')
            trip = (destination, master)
        
        origin_id = scraper.get_renfe_station_id(trip[0])
        destination_id = scraper.get_renfe_station_id(trip[1])
    
        # 15, 17, 19, 20, 21
        date = datetime.date(day=20, month=6, year=2024)
    
        buffer_df_trips, buffer_df_stops = scraper.scrape_trips(origin_id=origin_id, 
                                                                destination_id=destination_id, 
                                                                init_date=date)
        
        df_trips = pd.concat([df_trips, buffer_df_trips], ignore_index=True)
        df_stops = pd.concat([df_stops, buffer_df_stops], ignore_index=True)
        print(f"######## ROWS OF DF_TRIPS: {df_trips.shape[0]} ############")
        
print(df_stops.head())

From 60000 to 51405
Date:  2024-06-20
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=51405&AF=2024&MF=06&DF=20&SF=4&ID=s
######## ROWS OF DF_TRIPS: 3 ############
From 51405 to 60000
Date:  2024-06-20
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=51405&D=MADRI&AF=2024&MF=06&DF=20&SF=4&ID=s
######## ROWS OF DF_TRIPS: 6 ############
From 60000 to 54413
Date:  2024-06-20
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=54413&AF=2024&MF=06&DF=20&SF=4&ID=s
######## ROWS OF DF_TRIPS: 19 ############
From 54413 to 60000
Date:  2024-06-20
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=54413&D=MADRI&AF=2024&MF=06&DF=20&SF=4&ID=s
######## ROWS OF DF_TRIPS: 30 ############
From 60000 to 05000
Date:  2024-06-20
Search url:  https://horarios.renfe.com/HIRRenfeWeb/buscar.do?O=MADRI&D=05000&AF=2024&MF=06&DF=20&SF=4&ID=s
##################################################
UNKNOWN STATION: loja
###########################

In [10]:
df_stops.head()

service_id stop_id  arrival  departure
0  02074_20-06-2024-07.30   60000        0          0
1  02074_20-06-2024-07.30   37200       57         59
2  02074_20-06-2024-07.30   37300       74         76
3  02074_20-06-2024-07.30   37704       99        101
4  02074_20-06-2024-07.30   50500      125        127

In [6]:
# Len of dataframe
print(len(df_stops))

194


In [7]:
# Remove rows with stop_id value '00000'
df_stops_new = df_stops[df_stops['stop_id'] != '00000']
print(len(df_stops_new))

192


In [8]:
# Group by service_id. 
grouped = df_stops_new.groupby("service_id")

# If first row of group has arrival and departure not equal to 0, then remove the group
for name, group in grouped:
    if group.iloc[0]['arrival'] != 0 and group.iloc[0]['departure'] != 0:
        df_stops_new = df_stops_new[df_stops_new['service_id'] != name]

print(len(df_stops_new))

192


In [9]:
# Print number of groups
print(len(grouped))

36


In [51]:
for name, group in grouped:
    if group.iloc[-1]['arrival'] != group.iloc[-1]['departure']:
        df_stops_new = df_stops_new[df_stops_new['service_id'] != name]

print(len(df_stops_new))

782


In [52]:
# If len of group is less than 2, then remove the group
for name, group in grouped:
    if len(group) < 2:
        df_stops_new = df_stops_new[df_stops_new['service_id'] != name]

print(len(df_stops_new))

782


In [53]:
# Save dataframe to csv
df_stops_new.to_csv('stops_HSR_Spain_20_April_24.csv', index=False)

In [10]:
# Save dataframe to csv
df_trips.to_csv('trips_HSR_Spain_15_April_24.csv', index=False)

In [11]:
import pandas as pd

dtypes = {'stop_id': str}
df_stops = pd.read_csv('stops_HSR_Spain_15_April_24.csv', dtype=dtypes)
df_stops.head()

service_id stop_id  arrival  departure
0  06301_15-04-2024-06.15   60000        0          0
1  06301_15-04-2024-06.15   71801      150        150
2  03063_15-04-2024-06.30   60000        0          0
3  03063_15-04-2024-06.30   70600       55         56
4  03063_15-04-2024-06.30   04040       81         82

In [11]:
# Len of dataframe
print(len(df_stops))

6241


In [12]:
print(set(df_stops['stop_id']))

{'04040', '71500', '79300', '04307', '70600', '70200', '60000', '00000', '71801', '78400'}


In [13]:
print(set(scraper.stations_df['stop_id']))

{'64100', '14100', '79309', '15211', '50500', '37704', '70101', '03410', '65304', '23021', '81100', '92102', '03213', '50417', '51400', '11014', '65318', '37606', '54413', '06006', '78301', '30100', '62002', '15410', '61307', '15100', '31200', '55020', '20309', '22300', '10400', '78200', '81202', '12100', '60000', '14223', '11208', '51300', '71801', '04040', '03309', '60905', '05000', '69110', '60902', '04307', '02002', '35206', '40100', '78400', '20300', '74200', '35400', '60911', '60400', '15009', '37200', '10600', '23004', '51003', '79315', '50300', '31400', '08223', '70600', '66100', '00000', '02003', '43019', '31412', '37300', '70200', '30200', '11200', '21010', '61200', '67200', '71400', '37500', '20200', '56312', '80100', '50102', '79300', '71500', '54400', '22100', '11511', '82100', '40008', '13200', '65000', '51405', '10500', '60600', '03100', '65300'}


In [14]:
# Get set of stations which are NOT in the column renfe_id of scraper.stations_df
print(set(df_stops['stop_id']) - set(scraper.stations_df['stop_id']))

set()


In [8]:
# remove rows in wich stop_id is not in the column renfe_id of scraper.stations_df
df_stops_clean = df_stops[df_stops['stop_id'].isin(scraper.stations_df['renfe_id'])]
df_stops_clean.reset_index(drop=True, inplace=True)
print(df_stops_clean)

                   service_id stop_id  arrival  departure
0      03063_01-03-2024-06.30   70600       55         56
1      03073_01-03-2024-07.30   78400      125        127
2      03093_01-03-2024-09.30   78400      119        121
3      03093_01-03-2024-09.30   79300      233        235
4      03093_01-03-2024-09.30   04307      250        250
...                       ...     ...      ...        ...
13673  05903_31-03-2024-20.01   03309       81         83
13674  05903_31-03-2024-20.01   60600      115        117
13675  05217_31-03-2024-21.04   00000        0          0
13676  05217_31-03-2024-21.04   03309       19         21
13677  05217_31-03-2024-21.04   60600       53         55

[13678 rows x 4 columns]


In [10]:
# save dataframe to csv
df_stops_clean.to_csv('stops_HSR_Spain_March_24_clean.csv', index=False)

In [15]:
result_dict = {}

grouped = df_stops.groupby("service_id")

for name, group in grouped:
    sub_dict = {}
    for index, row in group.iterrows():
        sub_dict[row['stop_id']] = (row['arrival'], row['departure'])
    result_dict[name] = sub_dict

print(result_dict)

{'03062_01-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)}, '03062_03-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)}, '03062_04-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)}, '03062_05-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)}, '03062_06-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)}, '03062_07-06-2024-05.50': {'71801': (0, 0), '71500': (33, 35), '78400': (60, 62), '04040': (103, 106), '70600': (130, 131), '70200': (169, 170), '60000': (200, 200)

In [16]:
trips = list(set(tuple(service.keys())for service in result_dict.values()))
trips

[('04307', '79300', '71801', '04040', '60000'),
 ('60000', '04040', '78400', '71500', '71801', '79300', '04307'),
 ('60000', '70200', '04040', '71500', '71801', '79300', '04307', '00000'),
 ('60000', '04040', '71500', '71801'),
 ('04307', '79300', '71801', '71500', '78400', '04040', '70600', '60000'),
 ('71801', '71500', '78400', '04040', '70200', '60000'),
 ('60000', '71801', '79300', '04307'),
 ('60000', '70600', '04040', '71801'),
 ('60000', '04040', '71801', '79300', '04307'),
 ('71801', '71500', '78400', '04040', '70600', '70200', '60000'),
 ('60000', '70200', '04040', '78400', '71500', '71801'),
 ('00000', '04307', '79300', '71801', '71500', '04040', '60000'),
 ('71801', '71500', '78400', '04040', '70600', '60000'),
 ('60000', '04040', '78400', '71801', '79300', '04307'),
 ('71801', '60000'),
 ('60000',
  '70200',
  '70600',
  '04040',
  '78400',
  '71500',
  '71801',
  '79300',
  '04307'),
 ('04307', '79300', '71801', '60000'),
 ('60000', '70600', '04040', '71801', '79300', '043

In [17]:
# Initialize line with max length trip
line_stations = list(trips.pop(trips.index(max(trips, key=len))))

# Complete corridor with other stops that are not in the initial defined corridor
for trip in trips:
    for i, station in enumerate(trip):
        if station not in line_stations:
            # If station is the last one, append it to the end of the corridor
            if i == len(trip) - 1:
                line_stations.append(station)
            else:
                # If station is not the last one, insert it in the corridor before the next station
                index = line_stations.index(trip[i + 1])
                line_stations.insert(index, station)

print(line_stations)

['60000', '70200', '70600', '04040', '78400', '71500', '71801', '79300', '04307', '00000']


In [18]:
mapped_names = scraper.stations_df.set_index('stop_id')['stop_name'].to_dict()
line_stations_names = list(map(mapped_names.get, line_stations))

print(line_stations_names)

['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', 'Lleida', 'Tarragona', 'Barcelona-Sants', 'Girona', 'Figueres Vilafant', 'Unknown']


In [ ]:
# ['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', 'Tardienta', 'Huesca']

['Madrid-Puerta de Atocha', 'Guadalajara', 'Calatayud', 'Zaragoza-Delicias', [['Tarragona', 'Lleida', 'Barcelona-Sants', 'Girona', 'Figueres Vilafant'], ['Tardienta', 'Huesca']]]

In [ ]:
['60000', '70200', '70600', '04040'], [['71500', '78400', '71801', '79300', '04307'], ['78200', '74200']]

In [19]:
df_stations = scraper.stations_df.copy()

print(df_stations.head())

  stop_id            stop_name renfe_id   stop_lat  stop_lon
0   00000              Unknown    00000   0.000000  0.000000
1   31412            A Corunya    31412  43.352761 -8.409755
2   60911     Alicante/alacant    60911  38.344450 -0.495053
3   60600  Albacete-Los Llanos    60600  38.999384 -1.848450
4   60400  Alcazar de San Juan    60400  39.395628 -3.205744


In [22]:
estaciones_noreste = ['60000', '70200', '70600', '04040', '71500', '78400', '71801', '79300', '04307', '78200', '74200']

df_stations = df_stations[df_stations['stop_id'].isin(estaciones_noreste)]
df_stations.drop(columns=['renfe_id'], inplace=True)
df_stations.reset_index(drop=True, inplace=True)
print(df_stations)

   stop_id                stop_name   stop_lat  stop_lon
0    70600                Calatayud  41.346692 -1.638680
1    04307        Figueres Vilafant  42.264771  2.943547
2    79300                   Girona  41.979303  2.817006
3    74200                   Huesca  42.133594 -0.409745
4    78400                   Lleida  41.620696  0.632669
5    60000  Madrid-Puerta de Atocha  40.406442 -3.690886
6    71500                Tarragona  41.111624  1.253214
7    04040        Zaragoza-Delicias  41.658649 -0.911615
8    71801          Barcelona-Sants  41.379220  2.140624
9    70200              Guadalajara  40.644103 -3.182230
10   78200                Tardienta  41.975751 -0.538314


In [23]:
# save dataframe to csv
df_stations.to_csv('estaciones_corredor_noreste.csv', index=False)